# Loading from Excel after creating a volume folder

Remember that to create a volume folder to store files and get their file path, we should follow the following instructions:
1. Go to Catalog>Add data (+) or in the sidebar go to Data Engineering>Data Ingestion> Files> Upload files to a volume> Create volume inside your catalog:
![](path)

In [0]:
%pip install openpyxl
import pandas as pd
import openpyxl

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
df = pd.read_excel("/Volumes/workspace/test_schema/test_volume_folder/ufo_sighting_data.xlsx", engine='openpyxl')
sdf = spark.createDataFrame(df)

In [0]:
sdf.show()

+----------------+--------------------+--------------+-------+---------+---------------------------+-------------------------------+--------------------+---------------+----------+------------+
|       Date_time|                city|state/province|country|UFO_shape|length_of_encounter_seconds|described_duration_of_encounter|         description|date_documented|  latitude|   longitude|
+----------------+--------------------+--------------+-------+---------+---------------------------+-------------------------------+--------------------+---------------+----------+------------+
|10/10/1949 20:30|          san marcos|            tx|     us| cylinder|                       2700|                     45 minutes|This event took p...|      4/27/2004|29.8830556| -97.9411111|
|10/10/1949 21:00|        lackland afb|            tx|   NULL|    light|                       7200|                        1-2 hrs|1949 Lackland AFB...|     12/16/2005|  29.38421|  -98.581082|
|10/10/1955 17:00|chester (uk/

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType
import pandas as pd

schema = StructType([
    StructField("Date_time", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state/province", StringType(), True),
    StructField("country", StringType(), True),
    StructField("UFO_shape", StringType(), True),
    StructField("length_of_encounter_seconds", FloatType(), True),
    StructField("described_duration_of_encounter", StringType(), True),
    StructField("description", StringType(), True),
    StructField("date_documented", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True)
])

# Read CSV with proper handling of mixed types
df_csv = pd.read_csv("/Volumes/workspace/test_schema/test_volume_folder/ufo_sighting_data.csv", 
                     low_memory=False, 
                     dtype=str)  # Read everything as string first

# Convert problematic columns with error handling
df_csv['length_of_encounter_seconds'] = pd.to_numeric(df_csv['length_of_encounter_seconds'], errors='coerce')
df_csv['latitude'] = pd.to_numeric(df_csv['latitude'], errors='coerce')
df_csv['longitude'] = pd.to_numeric(df_csv['longitude'], errors='coerce')

# Handle NaN values that cause Arrow conversion issues
df_csv = df_csv.fillna({
    'length_of_encounter_seconds': 0.0,
    'latitude': 0.0,
    'longitude': 0.0
})

# Create Spark DataFrame
sdf_csv = spark.createDataFrame(df_csv, schema=schema)
sdf_csv.show()

+----------------+--------------------+--------------+-------+---------+---------------------------+-------------------------------+--------------------+---------------+----------+------------+
|       Date_time|                city|state/province|country|UFO_shape|length_of_encounter_seconds|described_duration_of_encounter|         description|date_documented|  latitude|   longitude|
+----------------+--------------------+--------------+-------+---------+---------------------------+-------------------------------+--------------------+---------------+----------+------------+
|10/10/1949 20:30|          san marcos|            tx|     us| cylinder|                     2700.0|                     45 minutes|This event took p...|      4/27/2004|29.8830556| -97.9411111|
|10/10/1949 21:00|        lackland afb|            tx|   NULL|    light|                     7200.0|                        1-2 hrs|1949 Lackland AFB...|     12/16/2005|  29.38421|  -98.581082|
|10/10/1955 17:00|chester (uk/

In [0]:
sdf_us = sdf.filter(sdf['country'] == 'us')
sdf_us.show()

+----------------+----------------+--------------+-------+---------+---------------------------+-------------------------------+--------------------+---------------+----------+------------+
|       Date_time|            city|state/province|country|UFO_shape|length_of_encounter_seconds|described_duration_of_encounter|         description|date_documented|  latitude|   longitude|
+----------------+----------------+--------------+-------+---------+---------------------------+-------------------------------+--------------------+---------------+----------+------------+
|10/10/1949 20:30|      san marcos|            tx|     us| cylinder|                       2700|                     45 minutes|This event took p...|      4/27/2004|29.8830556| -97.9411111|
|10/10/1956 21:00|            edna|            tx|     us|   circle|                         20|                       1/2 hour|My older brother ...|      1/17/2004|28.9783333| -96.6458333|
|10/10/1960 20:00|         kaneohe|            hi|

# Save the US dataframe into a delta table

In [0]:
sdf_us.write.format("delta").mode("overwrite").saveAsTable("test_schema.ufo_sighting_data")

# Filter original dataset for Great Britain dataset

In [0]:
# Filter from sdf only when country = 'gb'
sdf_gb = sdf.filter(sdf['country'] == 'gb')
sdf_gb.show()

+----------------+--------------------+--------------+-------+---------+---------------------------+-------------------------------+--------------------+---------------+---------+---------+
|       Date_time|                city|state/province|country|UFO_shape|length_of_encounter_seconds|described_duration_of_encounter|         description|date_documented| latitude|longitude|
+----------------+--------------------+--------------+-------+---------+---------------------------+-------------------------------+--------------------+---------------+---------+---------+
|10/10/1955 17:00|chester (uk/england)|          NULL|     gb|   circle|                         20|                     20 seconds|Green/Orange circ...|      1/21/2008|     53.2|-2.916667|
|10/10/1965 21:00|  penarth (uk/wales)|          NULL|     gb|   circle|                        180|                   about 3 mins|penarth uk  circl...|      2/14/2006|51.434722|    -3.18|
|10/10/1974 21:30|  cardiff (uk/wales)|          N

In [0]:
# append sdf_gb to the delta table
sdf_gb.write.format("delta").mode("append").saveAsTable("test_schema.ufo_sighting_data")

# Save other countries

In [0]:
# from sdf add the all other different than us and gb into the delta table
sdf_others = sdf_csv.filter((sdf_csv['country'] != 'us') & (sdf_csv['country'] != 'gb'))
sdf_others.show()

+----------------+--------------------+--------------+-------+---------+---------------------------+-------------------------------+--------------------+---------------+----------+-----------+
|       Date_time|                city|state/province|country|UFO_shape|length_of_encounter_seconds|described_duration_of_encounter|         description|date_documented|  latitude|  longitude|
+----------------+--------------------+--------------+-------+---------+---------------------------+-------------------------------+--------------------+---------------+----------+-----------+
|10/10/1994 23:00|toronto (greater ...|            on|     ca|   sphere|                     3600.0|                        ~1 hour|  Large rusty sphere|       7/3/2013| 43.666667| -79.416667|
|10/10/1998 22:30|st. john&#39s (ca...|            nf|     ca|      egg|                     7200.0|                        2 hours|Started off as 3 ...|      12/2/2000|     47.55| -52.666667|
|10/10/2000 07:30|   victoria (cana

In [0]:
# Get the schema of the existing table
existing_table_schema = spark.table("test_schema.ufo_sighting_data").schema

# Cast the DataFrame to match the schema of the existing table
sdf_others_casted = sdf_others.select(
    [sdf_others[col.name].cast(col.dataType) for col in existing_table_schema]
)

# Write the DataFrame to the Delta table
sdf_others_casted.write.format("delta").mode("append").saveAsTable("test_schema.ufo_sighting_data")